# Capstone Project

# The Battle of Neighborhoods - Part 1

**Contents**

For this week, you will required to submit the following:

1. A description of the problem and a discussion of the background. (15 marks)
1. A description of the data and how it will be used to solve the problem. (15 marks)


**1. A description of the problem and a discussion of the background.**

Police forces face a problem of limited resources and where to apply them. If they could gain an insight into where higher crime rates are likely to occur then a more efficient approach could be achieved.
    
In this capstone we will look at a combination of crime data and venues to see if there is a statistical/ML method of comparing neighborhoods and whether a particular cluster of venues implies a high crime rate.

Also, we shall examine 2 areas, the city of Reading and the city of Oxford. Both of these are cities in the south of England and contain a large university. Oxford has a population of approx 150,00 with Reading just over 210,000.

The ultimate aim and indeed measure of usefulness of this study is to help law enforcement to focus on particular crime types and be able to apply resources more efficiently to a particular area. To keep the size of this study manageable only the following questions will be investigated.

1. Are there similarities in the kinds of venues at or near to a crime hot spot, irrespective of the type of crime?
1. Focussing on shoplifting, are particular types of venues subject to shoplifting crimes?
1. "Shoplifting peaks between 3 p.m. and 4 p.m., and is lowest from 6 a.m. and 7 a.m." - is this true?
1. Is there a set of types of crime and of venues which indicate higher crime activity.

Across a country any area can be portioned into a neighborhood. For the purposes of this Capstone we regard a neighbourhood to cover a few miles in width. For the 2 area in consideration, Reading and Oxford, we shall center on the main shopping/retail/late night entertainment region of each.

A neighborhood will contain areas with differing crime levels. High crime is typically not spread over an area but will be concentrated into so called 'hot-spots'. If we can identify these hot-spots then we can look into whether the kinds of venues in the vicinity of that hot-spot such as shops, bars, cinemas etc indicate a likelihood of crime occurring there in future.

**2. A description of the data and how it will be used to solve the problem.**

In this study there are two groups of data. One is the crime data with location details and the other is the venue information also with location details.

## Data 1 - Crime Data

Here we combine spatial data regarding police neighborhoods, given in JSON format of latitude longitude coordinates, with crime data that also has lat long coordinates.

#### Police Neighborhood spatial data

JSON list of latitude, longitude coordinates defining the boundary of the neighborhood.

https://data.police.uk/data/

Firstly, get the list of police forces in England.

The end point does not require any parameters. 
The returned JSON is in the form of a list with each list element being "id":"&lt;force id>", "name":"&lt;force name>".

| Endpoint  | Return Type | Example |
| ------------- | ------------- | ---|
| https://data.police.uk/api/forces  | JSON  | <pre>[ {"id":"bedfordshire","name":"Bedfordshire Police"},</pre> |


Then get the neighborhoods for a police force, (use the force id - my apologies to Star Wars fans).
The endpoint requires the force id from the previous step, e.g. bedfordshire.
The returned JSON is in the form of a list with each list element being "id":"&lt;neighborhood id>", "name":"&lt;neighborhood name>".

| Endpoint | Return Type | Example |
| ------------- | ------------- | ---|
| https://data.police.uk/api/bedfordshire/neighbourhoods | JSON | <pre>[{"id":"BD1","name":"Ampthill,Flitwick,Silsoe"},</pre> |

and finally, get the boundary in latitude, longitude pairs for that neighborhood (use the neighborhood id).
The endpoint requires the force id from the first step, e.g. bedfordshire, and the neighborhood id from the previous step, e.g. BD2.
The returned JSON is in the form of a list with each list element being "latitude": latitude of boundary vertex, "longitude": longitude of boundary vertex.

| Endpoint  | Return Type | Example |
| ------------- | ------------- | ---|
| https://data.police.uk/api/bedfordshire/BD2/boundary | JSON | <pre>[{"latitude":"52.200075375","longitude":"-0.548369535"}, </pre>|

#### Crime data per police force

A police force is a collection of neighborhoods. The crime data shows the type of crime, time it took place and the latitude, longitude co-ordinates within that police force area. The crime data is split by month into separate csv files.

The data is not split into neighborhoods but is for the whole police force. The boundaries serve to give us a way to filter the data into more manageable data sizes.

https://data.police.uk/data/

![alt text](CrimeData.png "Police Crime Data - Main Page")

It is also possible to query the police API for the same data within a polygon region and for a specific month.

## Data 2 - Venue Data

#### Foursquare venues

A coordinate can be supplied to the Foursquare API to obtain the venues within a specified radius. Each venue belongs to particular category and also, to a specific level of category. Out of the information per venue we only require the venue category.

The endpoint details are as follows. The *client id* and *client secret* are specific to the user and require a new user to sign up to Foursquare. *ll* is latitude longitude separated by a comma. Version  has been set here to "20180604" - see the Foursquare API site for more information. *MYRADIUS* will be set to the distance from the centroid of the cell from which to gather the venue information, typically set to a multiple of the cell size, e.g. cell size = 100 metres, MYRADIUS = 500 metres. *limit* can go up to 50 (per the latest version of the docs), we will use 50 in our study.

https://api.foursquare.com/v2/venues/search?client_id=&lt;clientid>&client_secret=&lt;clientsecret>&ll=40.7149555,-74.0153365&v=20180604&radius=MYRADIUS&limit=30


Foursquare API :
https://developer.foursquare.com/docs

#### Foursquare venue categories

A list of categories is also supplied by the Foursquare API.

These are at specified levels. For example, if a restaurant has a category of "Japanese Restaurant" and another one "Asian Restaurant" then they are both under the same general category of Food (Level 0) and Asian Restaurant (Level 1) but the venue information has given us a name that is deeper in the hierarchy (Japanese Restaurant is Level 2). We 'normalise' each venue category into levels. So for the two restaurants just mentioned we might look at Level 0, both Food. Level 1, both Asian Restaurant. Level 2, one is Japanese Restaurant and the other is Not-categorized (my terminology). Note that the level numbers I give here and use in the study are my own invention - Foursquare does not label their data in this way.

<pre>
Level 0 : Food
├── Level 1 : Afghan Restaurant
├── Level 1 : African Restaurant
│   └── Level 2 : Ethiopian Restaurant
├── ...
├── Level 1 : Asian Restaurant
│   ├── ...
│   ├── Level 2 : Japanese Restaurant
│   ├── ...
│   ├── Level 2 : Thai Restaurant
│       └── Level 3 : Som Tum Restaurant
│   └── ...
├── ...
</pre>

Page describing the Foursquare categories

https://developer.foursquare.com/docs/resources/categories

Page describing the API call

https://developer.foursquare.com/docs/api/venues/categories

API endpoint to receive JSON list of categories

https://api.foursquare.com/v2/venues/categories


Levels 0 and 1 should provide the most that we need but keep the others in mind in case your use differs.

## How the data is combined.

### A. Identify the crime hotspots in a neighborhood.

A.1. Gridify the neighborhood into square cells, e.g. 100 meters to a side.
    
    Get police forces.
    
    Get neighborhoods for a police force.
    
    Get neighborhood boundary.
    
    Gridify the neighborhood into cells.
    
A.2. For each cell, count the number of crimes that occured within a certain distance of the centroid, e.g. within 500 meters.

    Get the crime data for the whole police force
    
    [Optional] Filter the police force data into a neighborhood.
    
    Filter the crimes to those within the search area (cell centroid + search distance).

A.3. Apply statistical measures to establish significance of the crime levels - is this really a hot spot or just a bit higher than its neighbouring cells.

### B. Do venues influence the crime rates nearby

B.1. For each hotspot get the venues nearby

B.2. Segment/cluster the venues - perhaps by differing levels.

B.3. Compare the results from two neighborhoods, not necessarily from the same police force.
